<a href="https://colab.research.google.com/github/Dimisz/tensorflow_lectures/blob/main/RNN_lecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>